# Bottle Data

### Purpose
This notebook provides a quick outline and example of working with OOI Discrete Summary Spreadsheet bottle data. It includes how to load it, how to parse data quality flags, how to derive some important values, and work with the data to make it easy to use for data validation and comparison purposes.

### Background
The Ocean Observatories Initiative (OOI) is an NSF-funded project for long-term (25 years) infrastructure for ocean observations. There are three main elements of OOI: (1) The Regional Cabled Array off the coast of Washington; (2) the Endurance Array on the Oregon-Washington shelf; and (3) the Coastal & Global Scale Nodes with mooring arrays in the Irminger Sea (Irminger Sea Array), at Ocean Station Papa (Papa Array), in the Southern Ocean and Argentine Basin (descoped and no longer collecting data), and at the New England Shelf (Pioneer Array-NES).

OOI deploys a variety of instrumentation that aim to measure the physical, chemical, and biological components of ocean activity. For data validation and comparison purposes with instrumentation, OOI also performs CTD casts and collects Niskin bottles for laboratory analysis during deployment and recovery of each mooring at each instrument depth. The laboratory analysis include:
* Oxygen
* Salinity
* Nutrients (Nitrate, Nitrite, Ammonium, Phosphate, Silicate)
* Chlorophyll
* Carbon System (at least two of DIC/TA/pH/pCO2)

### Data
The cruise data is hosted on OOI's Alfresco document server at https://alfresco.oceanobservatories.org, hereafter reffered to as Alfresco. Cruise data is hosted under OOI > {Array} > Cruise Data > {Cruise}. Cruise data includes the following key pieces of information: Cruise Reports; Ship Data (e.g. sonars, underway instruments, etc.); CTD; and Water Sampling Data.

We are principally concerned with the Water Sampling data, which can be found under {Cruise} > Ship Data > Water Sampling. The Water Sampling folder contains both digitized and scanned CTD logs as well as the results from the labs for each measured parameter. Helpfully, the CTD Bottle Data (.btl files) have been merged with the laboratory results and the CTD Sampling Logs to create **Discrete Summary Spreadsheet** of the water sampling data. These are the best place to start when working with OOI Bottle Data and are the data sources that we will be using in this example notebook.

In [1]:
import os, sys, re
import numpy as np
import pandas as pd
import PyQt5
import warnings
warnings.filterwarnings("ignore")

In [2]:
sys.path.append("/home/areed/Documents/OOI/reedan88/ooinet/")
from ooinet.Bottles import *

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

---
### Niskin & Discrete Bottle Sampling
These are discrete water samples collected via Niskin Bottle casts during deployment and recovery of the moored instrumentation. The data is downloaded from OOI Alfresco website as excel files. Parameters sampled are:
* Oxygen
* Salinity
* Nutrients: phosphate, nitrate, nitrite, ammonium, silicate
* Chlorophyll
* Carbon system: Dissolved inorganic carbon, total alkalinity, pH

First, we're going to load, clean, and process the discrete water sampling.

In [4]:
bottle_data = pd.read_excel("../data/bottle_data/Irminger_Sea-02_AT30-01_Discrete_Summary.xlsx")
bottle_data.head()

Cruise  Station  Target Asset  Start Latitude [degrees]  \
0  AT30-01        1  release test                 42.547833   
1  AT30-01        1  release test                 42.547833   
2  AT30-01        1  release test                 42.547833   
3  AT30-01        1  release test                 42.547833   
4  AT30-01        1  release test                 42.547833   

   Start Longitude [degrees]          Start Time [UTC]  Cast  \
0                    -62.584  2015-08-07T12:39:53.000Z     1   
1                    -62.584  2015-08-07T12:39:53.000Z     1   
2                    -62.584  2015-08-07T12:39:53.000Z     1   
3                    -62.584  2015-08-07T12:39:53.000Z     1   
4                    -62.584  2015-08-07T12:39:53.000Z     1   

           Cast Flag  Bottom Depth at Start Position [m]  \
0  *0000000000000100                                1723   
1  *0000000000000100                                1723   
2  *0000000000000100                                1723   
3  *0000000000000100                                1723   
4  *0000000000000100                                1723   

                   CTD File  ... Discrete pH Replicate Flag  \
0  C:\data\ctd\at30_001.hex  ...                   -9999999   
1  C:\data\ctd\at30_001.hex  ...                   -9999999   
2  C:\data\ctd\at30_001.hex  ...                   -9999999   
3  C:\data\ctd\at30_001.hex  ...                   -9999999   
4  C:\data\ctd\at30_001.hex  ...                   -9999999   

   Calculated Alkalinity [umol/kg] Calculated DIC [umol/kg]  \
0                         -9999999                 -9999999   
1                         -9999999                 -9999999   
2                         -9999999                 -9999999   
3                         -9999999                 -9999999   
4                         -9999999                 -9999999   

  Calculated pCO2 [uatm]  Calculated pH Calculated CO2aq [umol/kg]  \
0               -9999999       -9999999                   -9999999   
1               -9999999       -9999999                   -9999999   
2               -9999999       -9999999                   -9999999   
3               -9999999       -9999999                   -9999999   
4               -9999999       -9999999                   -9999999   

   Calculated Bicarb [umol/kg]  Calculated CO3 [umol/kg]  Calculated Omega-C  \
0                     -9999999                  -9999999            -9999999   
1                     -9999999                  -9999999            -9999999   
2                     -9999999                  -9999999            -9999999   
3                     -9999999                  -9999999            -9999999   
4                     -9999999                  -9999999            -9999999   

   Calculated Omega-A  
0            -9999999  
1            -9999999  
2            -9999999  
3            -9999999  
4            -9999999  

[5 rows x 80 columns]

Replace the fill value of ```-9999999``` with NaNs. Some columns were imported as floats and others as strings or objects, so this needs to be done twice:

In [5]:
bottle_data = bottle_data.replace(to_replace="-9999999", value=np.nan)
bottle_data = bottle_data.replace(to_replace=-9999999, value=np.nan)

Convert times from strings to pandas datetime objects and remove the timezone:

In [6]:
bottle_data["Start Time [UTC]"] = bottle_data["Start Time [UTC]"].apply(lambda x: convert_times(x))
bottle_data["CTD Bottle Closure Time [UTC]"] = bottle_data["CTD Bottle Closure Time [UTC]"].apply(lambda x: convert_times(x))

Some of the data returned from the labs which process samples contain non-numeric indicators for if data were below detection threshold. These entries typically have a "less than" sign. Convert any values with a "<", which indicates a value not statistically significant from zero, with zero:

In [7]:
bottle_data = bottle_data.applymap(not_statistically_sigificant)

#### Filter data
Next, we want to filter the data based on the data flag associated with the sample. First, we should drop all of the ```NaN``` values in the Niskin column to eliminate entries without any bottle samples. While these CTD casts may be of interest, the bottle data does not have any data of which we can make use of.

In [8]:
bottle_data = bottle_data.dropna(subset=["Niskin/Bottle Position"])

#### Interpret Flags
Next, we want to parse the Data Quality Flags in the bottle data. 

We intrept the data into the standard QARTOD flaging scheme of:
* 1 = good
* 2 = not run
* 3 = suspect
* 4 = bad
* 9 = missing

The "Replicate Flags" are reduced into a boolean value indicating that either there is a replicate sample or not. By replicate, we mean if there are more than one sample collected from a depth, e.g. two Niskins were closed at a particular depth and oxygen sampled from both. It does *not* mean that a particular analysis was repeated on the same sample multiple times, e.g. repeating nutrient analysis on a single Niskin to derive a final value.

In [9]:
for col in bottle_data.columns:
    if "Flag" in col:
        if "CTD" in col and "File" not in col:
            bottle_data[col] = bottle_data[col].apply(lambda x: interp_ctd_flag(x))
        elif "Discrete" in col:
            bottle_data[col] = bottle_data[col].apply(lambda x: interp_discrete_flag(x))
        elif "Replicate" in col:
            bottle_data[col] = bottle_data[col].apply(lambda x: interp_replicate_flag(x))
        elif "Niskin" in col:
            bottle_data[col] = bottle_data[col].apply(lambda x: interp_niskin_flag(x))
        else:
            pass

---
### Calculate Physical Properties
With the bottle samples cleaned up and the flags parsed into something simpler and easier to read, we next want to calculate the conservative seawater properties, such as density, conservative temperature, absolute salinity, etc., using the updated Thermodynamic Equations of State - 2010 (TEOS-10) algorithms. These are implemented with the ```gsw``` package. 

This step will allow us to correct for pressure and temperature effect between lab analysis and in-situ measurements, as well as allow for better comparison between depths and locations in the same array. 

In [10]:
import gsw

Get the measured parameters, taking the average if measured by more than one sensor

In [11]:
SP = bottle_data[["CTD Salinity 1 [psu]", "CTD Salinity 2 [psu]"]].mean(axis=1, skipna=True)
T = bottle_data[["CTD Temperature 1 [deg C]", "CTD Temperature 2 [deg C]"]].mean(axis=1, skipna=True)
P = bottle_data["CTD Pressure [db]"]
LAT = bottle_data["CTD Latitude [deg]"]
LON = bottle_data["CTD Longitude [deg]"]

Calculate the conservative temperature, absolute salinity, density, and potential density of the observations

In [12]:
# Absolute salinity
SA = gsw.conversions.SA_from_SP(SP, P, LON, LAT)
bottle_data["CTD Absolute Salinity [g/kg]"] = SA

# Conservative temperature
CT = gsw.conversions.CT_from_t(SA, T, P)
bottle_data["CTD Conservative Temperature"] = CT

# Density
RHO = gsw.density.rho(SA, CT, P)
bottle_data["CTD Density [kg/m^3]"] = RHO

# Calculate potential density
SIGMA0 = gsw.density.sigma0(SA, CT)
bottle_data["CTD Sigma [kg/m^3]"] = RHO

del SP, T, P, LAT, LON, SA, CT, RHO, SIGMA0

---
## Calculate Carbon System Parameters
The discrete water samples were tested for Total Alkalinity, Dissolved Inorganic Carbon, and pH [Total Scale]. We can calculate the discrete water sample pCO<sub>2</sub> concentrations from the TA and DIC using the ```CO2SYS``` program. Implementation in python is achieved using the PyCO2SYS package. Full documentation on how to install and use PyCO2SYS can be found at https://pyco2sys.readthedocs.io/en/latest/. 

For our purposes, we will be utilizing the following inputs:
* ```PAR1```: First measured carbon system measurement
* ```PAR2```: Second measured carbon system measurement
* ```PAR1_type```: The type of PAR1 
        * 1 = Total Alkalinity umol/kg
        * 2 = DIC umol/kg
        * 3 = pH Total Scale
        * 4 = pCO2
        * 5 = fCO2
* ```PAR2_type```: The type of PAR2


We will also use the following optional hydrographic inputs:
* ```salinity```: practical salinity
* ```temperature```: the temperature at which PAR1 and PAR2 are provided (in C)
* ```pressure```: the water pressure at which ```PAR1``` and ```PAR2``` are measured

Additionally, we will pass in some of the nutrient data that we collect:
* ```SIO4```: discrete silicate
* ```PO4```: discrete phosphate
* ```NH4```: discrete ammonia

The nutrient values need to be (1) converted to umol/kg from uM and (2) have NaNs filled in with 0, which is the default value.

In [13]:
import PyCO2SYS as pyco2

In [14]:
# Get the key parameters
DIC = bottle_data["Discrete DIC [umol/kg]"]
TA = bottle_data["Discrete Alkalinity [umol/kg]"]
PH = bottle_data["Discrete pH [Total scale]"]
SAL = bottle_data["Discrete Salinity [psu]"]

# Set the input hydrographic parameters at which the DIC/TA/pH lab measurements were performed
TEMP_IN = 25
PRES_IN = 0

# Get the hydrographic parameters at which the samples were taken
TEMP_OUT = bottle_data[["CTD Temperature 1 [deg C]","CTD Temperature 1 [deg C]"]].mean(axis=1, skipna=True)
PRES_OUT = bottle_data["CTD Pressure [db]"]
RHO = bottle_data["CTD Density [kg/m^3]"]

# Nutrient inputs = need to fill NaNs with zeros otherwise will return NaNs
SIO4 = bottle_data["Discrete Silicate [uM]"].fillna(value=0)
PO4 = bottle_data["Discrete Phosphate [uM]"].fillna(value=0)
NH4 = bottle_data["Discrete Ammonium [uM]"].fillna(value=0)

# Convert to umol/kg
SIO4 = SIO4*(1000/1)*(1/RHO)
PO4 = PO4*(1000/1)*(1/RHO)
NH4 = NH4*(1000/1)*(1/RHO)

Declare the parameter input types to pass into ```CO2SYS```

In [15]:
PAR1 = bottle_data['Discrete Alkalinity [umol/kg]']
PAR2 = bottle_data['Discrete DIC [umol/kg]']
PAR1TYPE = 1
PAR2TYPE = 2
PHSCALEIN = 1
K1K2CONSTANTS = 1
K2SO4CONSTANTS = 1

#### Calculate Carbon System
With all of the mandatory and optional carbon system parameters selected, we now calculate the carbon system for the bottle data

In [16]:
CO2dict = pyco2.CO2SYS(PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMP_IN, TEMP_OUT, PRES_IN, PRES_OUT, SIO4, PO4, PHSCALEIN, K1K2CONSTANTS, K2SO4CONSTANTS)

Now add the calculate carbon system parameters to the bottle data

In [17]:
bottle_data['Calculated Alkalinity [umol/kg]'] = CO2dict['TAlk']
bottle_data['Calculated CO2aq [umol/kg]'] = CO2dict['CO2out']
bottle_data['Calculated CO3 [umol/kg]'] = CO2dict['CO3out']
bottle_data['Calculated DIC [umol/kg]'] = CO2dict['TCO2']
bottle_data['Calculated pCO2 [uatm]'] = CO2dict['pCO2out']
bottle_data['Calculated pCO2 [lab conditions]'] = CO2dict['pCO2in']
bottle_data['Calculated pH [lab conditions]'] = CO2dict['pHinTOTAL']
derivedPH = pd.Series(data=CO2dict["pHoutTOTAL"], index=bottle_data.index)

### Assessment
We can check on how well ```CO2SYS``` reconstructed the carbon system by comparing the **Calculated pH**, derived from the DIC and TA, against the **Discrete pH**. This serves as an independent check and bound on the error introduced by the carbonate system algorithms. 

In [19]:
bottle_data[["Discrete pH [Total scale]","Calculated pH [lab conditions]"]].dropna()

Discrete pH [Total scale]  Calculated pH [lab conditions]
42                      7.7018                        7.729633
47                      7.8202                        7.844386
48                      7.8196                        7.884067
50                      7.8268                        7.841813
123                     7.8240                        7.836911
135                     7.7386                        7.754857
148                     7.8202                        7.827224
167                     7.7192                        7.726856
172                     7.8336                        7.823105
173                     7.8332                        7.815021
175                     7.8329                        7.824954

#### Adjust Discrete pH

Next, we need to adjust the **Discrete pH** to in-situ conditions. We'll do this by passing in the measured **Discrete pH** along with the **Discrete Alkalinity** and adjusting it to the in-situ conditions. To do this we only need to adjust the previous parameters such that ```PAR2``` is the **Discrete pH \[Total Scale\]** and ```PAR2TYPE``` is set to **3**.

In [20]:
PHout = CO2dict["pHinTOTAL"]
df = pd.DataFrame(data=[PH.values, PHout], index=["Measured", "CO2sys"]).T.dropna()
df = df[df["CO2sys"] > 7]
df = df.sort_values(by="Measured")
PHmeas = df["Measured"].values.reshape(-1,1)
PHcalc = df["CO2sys"].values.reshape(-1,1)

In [21]:
# Use sklearn linear regression model to determine the accuracy 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Fit a linear regression to the measured vs calculated pH values
regression = LinearRegression()
regression.fit(PHmeas, PHcalc)

# Get the regression values
PHpred = regression.predict(PHmeas)
PHmse = mean_squared_error(PHpred, PHcalc)
PHstd = np.sqrt(PHmse)

In [22]:
# Look at how closely the pH measurements match each other
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,8))
ax.scatter(PHmeas, PHcalc)
ax.plot(PHmeas, PHpred, c='r', label='Regression')

#ax.fill_between(PHmeas, PHpred-1.96*PHstd, PHpred+1.96*PHstd, color="tab:red")
ax.plot(PHmeas, PHpred-1.96*PHstd, ':', c='r', label='2 Stds')
ax.plot(PHmeas, PHpred+1.96*PHstd, ':', c='r')
ax.set_xlabel('Discrete pH\n[Total Scale, T=25$^{\circ}$C]', fontsize=12)
ax.set_ylabel('CO2SYS-calculated pH\n[Total Scale, T=25$^{\circ}$C]', fontsize=12)
ax.text(0.8,0.4,f'Std: {PHstd.round(3)}', fontsize='16', transform=ax.transAxes)
ax.legend(fontsize=12)
ax.grid()

In [23]:
regression.coef_, regression.intercept_

(array([[0.89931728]]), array([0.79766301]))

#### Adjust Discrete pH

Next, we need to adjust the **Discrete pH** to in-situ conditions. We'll do this by passing in the measured **Discrete pH** along with the **Discrete Alkalinity** and adjusting it to the in-situ conditions. To do this we only need to adjust the previous parameters such that ```PAR2``` is the **Discrete pH \[Total Scale\]** and ```PAR2TYPE``` is set to **3**.

In [24]:
PAR1 = bottle_data['Discrete Alkalinity [umol/kg]']
PAR2 = bottle_data['Discrete pH [Total scale]']
PAR1TYPE = 1
PAR2TYPE = 3

Recalculate the carbon system to get the in-situ pH

In [25]:
CO2dict = pyco2.CO2SYS(PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMP_IN, TEMP_OUT, PRES_IN, PRES_OUT, SIO4, PO4, PHSCALEIN, K1K2CONSTANTS, K2SO4CONSTANTS)

In [26]:
bottle_data["Calculated pH"] = CO2dict["pHoutTOTAL"]
# Fill in missing "Calculate pH" with the derived pH to increase the number of pH observations
bottle_data["Calculated pH"].fillna(value=derivedPH, inplace=True)
bottle_data[["Discrete pH [Total scale]","Calculated pH [lab conditions]","Calculated pH"]].dropna()

Discrete pH [Total scale]  Calculated pH [lab conditions]  Calculated pH
42                      7.7018                        7.729633       8.021301
47                      7.8202                        7.844386       8.092480
48                      7.8196                        7.884067       8.091827
50                      7.8268                        7.841813       8.093763
123                     7.8240                        7.836911       8.094680
135                     7.7386                        7.754857       8.047948
148                     7.8202                        7.827224       8.096595
167                     7.7192                        7.726856       8.023363
172                     7.8336                        7.823105       8.097088
173                     7.8332                        7.815021       8.097010
175                     7.8329                        7.824954       8.097150

---
### Save the Results
Finally, with the carbon system results fully derived, save the results to the data directory with the other bottle data (or wherever you want to save the data)

In [ ]:
bottle_data.to_csv("insert_path_here.csv")